# Imputación de Datos de indicadores con KNN

In [1]:
import pandas as pd
from sklearn.impute import KNNImputer

from ind import print_munprofile

Lectura del dataset

In [2]:
df = pd.read_csv('../data/processed/indicators_municipal.csv')
df.head()

,cve_mun,municipio,grad_margi,grad_rez_social,conectividad,servicios_basicos,elementos_salud,educacion,desigualdad,dependencia_economica,calidad_vivienda,seguridad_alimentaria,seguridad_social
0,20001,Abejones,Alto,Medio,Bajo,Bajo,Alto,Alto,Medio,Alto,Medio,Bajo,Alto
1,20002,Acatlán de Pérez Figueroa,Medio,Bajo,Medio,Bajo,Bajo,Medio,Medio,Medio,Medio,Medio,Medio
2,20003,Asunción Cacalotepec,Alto,Muy alto,Muy bajo,Muy bajo,Medio,Medio,Medio,Alto,Medio,Alto,Medio
3,20004,Asunción Cuyotepeji,Medio,Bajo,Bajo,Bajo,NaN,Medio,Medio,Alto,Medio,Medio,Medio
4,20005,Asunción Ixtaltepec,Bajo,Bajo,Medio,Bajo,Medio,Medio,Medio,Medio,Medio,Bajo,Medio


Columnas con datos faltantes

In [3]:
to_keep = ["elementos_salud", "desigualdad", "dependencia_economica"]
all_cols_missing = []

# show count of missing values per column
for col in df.columns:
    missing_count = df[col].isnull().sum()
    if missing_count > 0:
        print(f"{col}: {missing_count} missing values")
        all_cols_missing.append(col)

elementos_salud: 167 missing values
desigualdad: 4 missing values
dependencia_economica: 10 missing values


Mostrar perfil del municipio

In [4]:
muns = ["Matías Romero Avendaño", "Magdalena Mixtepec", "Asunción Cuyotepeji"]

In [5]:
for municipality_name in muns:
    print(f"\n\nPerfil del municipio antes de la imputación de datos:")
    print_munprofile(df, municipality_name)



Perfil del municipio antes de la imputación de datos:

================= PERFIL DEL MUNICIPIO: MATÍAS ROMERO AVENDAÑO =================

RESUMEN DEL PERFIL:
  • Marginación: Medio
  • Rezago social: Bajo
  • Conectividad: Medio
  • Servicios básicos: Bajo
  • Elementos de salud: Medio
  • Educación: Medio
  • Desigualdad: nan
  • Dependencia económica: Medio
  • Calidad de vivienda: Medio
  • Grado de acceso a alimentos: Bajo
  • Grado de acceso a servicios de salud: Bajo


Perfil del municipio antes de la imputación de datos:

=================== PERFIL DEL MUNICIPIO: MAGDALENA MIXTEPEC ===================

RESUMEN DEL PERFIL:
  • Marginación: Alto
  • Rezago social: Muy alto
  • Conectividad: Bajo
  • Servicios básicos: Muy bajo
  • Elementos de salud: Medio
  • Educación: Medio
  • Desigualdad: Medio
  • Dependencia económica: nan
  • Calidad de vivienda: Medio
  • Grado de acceso a alimentos: Medio
  • Grado de acceso a servicios de salud: Medio


Perfil del municipio antes de la

Quitar cve_mun y municipio

In [6]:
metadata = ["cve_mun", "municipio"]

df_cleaned = df.drop(columns=metadata)
df_cleaned.head()

,grad_margi,grad_rez_social,conectividad,servicios_basicos,elementos_salud,educacion,desigualdad,dependencia_economica,calidad_vivienda,seguridad_alimentaria,seguridad_social
0,Alto,Medio,Bajo,Bajo,Alto,Alto,Medio,Alto,Medio,Bajo,Alto
1,Medio,Bajo,Medio,Bajo,Bajo,Medio,Medio,Medio,Medio,Medio,Medio
2,Alto,Muy alto,Muy bajo,Muy bajo,Medio,Medio,Medio,Alto,Medio,Alto,Medio
3,Medio,Bajo,Bajo,Bajo,NaN,Medio,Medio,Alto,Medio,Medio,Medio
4,Bajo,Bajo,Medio,Bajo,Medio,Medio,Medio,Medio,Medio,Bajo,Medio


Ejemplos de valores faltantes para las columnas seleccionadas

In [7]:
# for each column in to_keep, show the first 3 rows with missing values
to_show = []
for col in to_keep:
    missing_rows = df_cleaned[df_cleaned[col].isnull()].head(3)
    if not missing_rows.empty:
        print(f"\nPrimeras 3 instancias con valores vacíos en '{col}':")
        # print(missing_rows[["municipio", col]])

        # imprima el nombre del municipio en df y el valor faltante
        for index, row in missing_rows.iterrows():
            print(f"Municipio: {df.loc[index, 'municipio']}, Valor faltante en '{col}': {row[col]}")

        to_show.append({
            "column": col,
            "rows": missing_rows.index
        })


Primeras 3 instancias con valores vacíos en 'elementos_salud':
Municipio: Asunción Cuyotepeji, Valor faltante en 'elementos_salud': nan
Municipio: Calihualá, Valor faltante en 'elementos_salud': nan
Municipio: Constancia del Rosario, Valor faltante en 'elementos_salud': nan

Primeras 3 instancias con valores vacíos en 'desigualdad':
Municipio: Matías Romero Avendaño, Valor faltante en 'desigualdad': nan
Municipio: San Francisco Chindúa, Valor faltante en 'desigualdad': nan
Municipio: Santa María Chimalapa, Valor faltante en 'desigualdad': nan

Primeras 3 instancias con valores vacíos en 'dependencia_economica':
Municipio: Magdalena Mixtepec, Valor faltante en 'dependencia_economica': nan
Municipio: San Andrés Tepetlapa, Valor faltante en 'dependencia_economica': nan
Municipio: San Francisco Logueche, Valor faltante en 'dependencia_economica': nan


Imputación de los datos

In [8]:
# label encode categorical columns
labels = {1: "Muy alto", 2: "Alto", 3: "Medio", 4: "Bajo", 5: "Muy bajo"}
labels_reverse = {v: k for k, v in labels.items()}

# codificar las columnas categóricas
for col in df_cleaned.columns:
    df_cleaned[col] = df_cleaned[col].map(labels_reverse).astype(float)

# imprimir 1er fila
df_cleaned.head()

,grad_margi,grad_rez_social,conectividad,servicios_basicos,elementos_salud,educacion,desigualdad,dependencia_economica,calidad_vivienda,seguridad_alimentaria,seguridad_social
0,2.0,3.0,4.0,4.0,2.0,2.0,3.0,2.0,3.0,4.0,2.0
1,3.0,4.0,3.0,4.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0
2,2.0,1.0,5.0,5.0,3.0,3.0,3.0,2.0,3.0,2.0,3.0
3,3.0,4.0,4.0,4.0,NaN,3.0,3.0,2.0,3.0,3.0,3.0
4,4.0,4.0,3.0,4.0,3.0,3.0,3.0,3.0,3.0,4.0,3.0


In [ ]:
# imput with KNNImputer
sub_df = df_cleaned[to_keep]
imputer = KNNImputer(n_neighbors=3)
imputed = imputer.fit_transform(sub_df)

# Re-add to df_cleaned
for i, col in enumerate(to_keep):
    df_cleaned[col] = imputed[:, i]

In [10]:
# ceil round for all columns in df_cleaned
for col in df_cleaned.columns:
    df_cleaned[col] = df_cleaned[col].round().astype(int)

Agregar columnas de metadatos

In [11]:
# add back metadata columns
df_final = pd.concat([df[metadata], df_cleaned], axis=1)

Mostrar los valores imputados

In [12]:
# return categorical columns to original labels
for col in df_cleaned.columns:
    df_final[col] = df_cleaned[col].map(labels).astype('category')

In [13]:
for item in to_show:
    col = item["column"]
    rows = item["rows"]
    print(f"\nValores imputados para '{col}':")
    print(df_final.loc[rows, ["municipio", col]])


Valores imputados para 'elementos_salud':
                 municipio elementos_salud
3      Asunción Cuyotepeji           Medio
10               Calihualá            Alto
19  Constancia del Rosario           Medio

Valores imputados para 'desigualdad':
                  municipio desigualdad
56   Matías Romero Avendaño       Medio
139   San Francisco Chindúa       Medio
406   Santa María Chimalapa       Medio

Valores imputados para 'dependencia_economica':
                  municipio dependencia_economica
47       Magdalena Mixtepec                  Alto
98     San Andrés Tepetlapa                  Alto
145  San Francisco Logueche                  Alto


Mostrar perfil de los municipios después de la imputación de datos

In [14]:
for municipality_name in muns:
    print(f"\n\nPerfil del municipio antes de la imputación de datos:")
    print_munprofile(df_final, municipality_name)



Perfil del municipio antes de la imputación de datos:

================= PERFIL DEL MUNICIPIO: MATÍAS ROMERO AVENDAÑO =================

RESUMEN DEL PERFIL:
  • Marginación: Medio
  • Rezago social: Bajo
  • Conectividad: Medio
  • Servicios básicos: Bajo
  • Elementos de salud: Medio
  • Educación: Medio
  • Desigualdad: Medio
  • Dependencia económica: Medio
  • Calidad de vivienda: Medio
  • Grado de acceso a alimentos: Bajo
  • Grado de acceso a servicios de salud: Bajo


Perfil del municipio antes de la imputación de datos:

=================== PERFIL DEL MUNICIPIO: MAGDALENA MIXTEPEC ===================

RESUMEN DEL PERFIL:
  • Marginación: Alto
  • Rezago social: Muy alto
  • Conectividad: Bajo
  • Servicios básicos: Muy bajo
  • Elementos de salud: Medio
  • Educación: Medio
  • Desigualdad: Medio
  • Dependencia económica: Alto
  • Calidad de vivienda: Medio
  • Grado de acceso a alimentos: Medio
  • Grado de acceso a servicios de salud: Medio


Perfil del municipio antes de